In [1]:
# conda install -c conda-forge jpype1==1.1
# conda install konlpy
# pip install konlpy 자바나 jpype 설치 필요없음
# nltk.download('punkt')
# conda install matplotlib
# import matplotlib

In [2]:
# matplotlib 사용 가능한지 테스트

# import matplotlib.pyplot as plt

# matplotlib.rcParams['axes.edgecolor'] = "black"
# matplotlib.rcParams['axes.facecolor'] = "white"

# plt.plot(["Seoul","Paris","Seattle"], [30,25,55])
# plt.xlabel('City')
# plt.ylabel('Response')
# plt.title('Experiment Result')
# plt.show()

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import nltk
# import matplotlib.pyplot as plt
# import seaborn as sns
import os
from konlpy.tag import Okt
from nltk.tokenize import word_tokenize
from random import seed
from random import randint
import re
from konlpy.tag import Okt

In [4]:
okt = Okt()

In [5]:
df = pd.read_csv('naver_shopping.txt', sep = "\t", header = None)
df = df.sample(frac=0.1)   # 학습시간때문에 10% 만 사용함
df.columns = ['label', 'text']

In [6]:
y = [0 if rate < 3 else 1 for rate in df.label] # 3점 미만이면 0(부정), 3점 초과이면 1(긍정)으로 라벨 생성

In [7]:
#평가 데이터셋 분리
X_train_val, X_test, y_train_val, y_test = train_test_split(df['text'].tolist(), y, random_state=0)

#학습, 검증 데이터셋분리
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, random_state=0)

print(len(X_train), len(X_val), len(X_test))
print(len(y_train), len(y_val), len(y_test))

11250 3750 5000
11250 3750 5000


In [8]:
# random_txt = randint(0, len(X_train))

# print('형태소 단위로 tokenize: ', okt.morphs(X_train[random_txt]))
# print('명사만 추출: ', okt.nouns(X_train[random_txt]))

# # morphs(text) : 텍스트에서 형태소를 반환한다
# # nouns(text) : 텍스트에서 명사를 반환한다
# # phrases(text) : 텍스트에서 어절을 뽑아낸다
# # pos(text) : 텍스트에서 품사 정보를 부착하여 반환한다. 

### 크롤링된 데이터 불러오기

In [9]:
forders = os.listdir('/home/adminuser/notebooks/modeling/raw_data/crawling/')

In [10]:
df_all = pd.DataFrame()

for i in range(0,len(forders)):
    if forders[i].split('.')[1] == 'csv':
        file = '/home/adminuser/notebooks/modeling/raw_data/crawling/'+forders[i]
        df= pd.read_csv(file,encoding='utf-8') 
        df_all = pd.concat([df_all, df])

In [11]:
df_all = df_all.dropna()

### Okt 명사만 사용 (tfidf1)

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

tfidf1 = TfidfVectorizer(tokenizer=okt.nouns, max_features=2000, min_df=5, max_df=0.5)

# min_df: 최소 빈도값을 설정
# DF는 특정 단어가 나타나는 '문서의 수'를 의미
# (단어 'home'의 경우 전체 문서에서 빈도는 4번 이지만,'home'이라는 단어가 포함된 문서의 수는 3개이기 때문에 DF = 3 )
# DF의 최소값을 설정하여 해당 값보다 작은 DF를 가진 단어들은 단어사전 (vocabulary_)에서 제외하고, 인덱스를 부여하지 않음
# 최소 DF가 5로 설정되었으니, 1,2,3,4인 것들은 모두 탈락하게 됩니다.

In [13]:
X_train_tfidf = tfidf1.fit_transform(X_train)

In [14]:
X_test_tfidf = tfidf1.transform(X_test)

In [15]:
# min_df_vectorizer = TfidfVectorizer(min_df = 2)
# min_df_vectorizer.fit(text)
# sorted(min_df_vectorizer.vocabulary_.items())
# from sklearn.feature_extraction.text import CountVectorizer
# sorted(X_train_tfidf.vocabulary_.items())
# print(X_test_tfidf)

In [25]:
clf1 = LogisticRegression(max_iter=1000)
results1= clf1.fit(X_train_tfidf, y_train)
print(results1)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [26]:
# train data 예측 정확도
print('#Train set score: {:.3f}'.format(clf.score(X_train_tfidf, y_train)))
# test data 예측 정확도
print('#Test set score: {:.3f}'.format(clf.score(X_test_tfidf, y_test)))

#Train set score: 0.820
#Test set score: 0.767


### Okt 명사, 동사, 형용사를 사용 (tfidf2)

In [28]:
def twit_tokenizer(text):
    target_tags = ['None', 'Verb', 'Adjective']
    result = []
    for word, tag in okt.pos(text, norm=True, stem=True):
        if tag in target_tags:
            result.append(word)
    return result

In [29]:
tfidf2 = TfidfVectorizer(tokenizer=twit_tokenizer, max_features=2000, min_df=5, max_df=0.5)

In [30]:
X_train_tfidf_twit = tfidf2.fit_transform(X_train)

In [31]:
X_test_tfidf_twit = tfidf2.transform(X_test)   

#fit 하지 않는다!! (참고: https://deepinsight.tistory.com/165)

In [32]:
clf2 = LogisticRegression(max_iter=1000)
results2 = clf2.fit(X_train_tfidf_twit, y_train)
print(results2)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)


In [33]:
# train data 예측 정확도
print('#Train set score: {:.3f}'.format(clf2.score(X_train_tfidf_twit, y_train)))
# test data 예측 정확도
print('#Test set score: {:.3f}'.format(clf2.score(X_test_tfidf_twit, y_test)))

#Train set score: 0.843
#Test set score: 0.827


### sample data에 적용

In [56]:
tmp= df_all.sample(10)
tmp["regrex"]=tmp.comment.map(lambda x : re.compile('[^ A-Za-z0-9ㄱ-ㅣ가-힣]+').sub('', str(x)))

In [57]:
random_txt_tfidf = tfidf1.transform(tmp['regrex'])
clf1.predict(random_txt_tfidf)

pred1 = pd.DataFrame({'clf1_label': clf1.predict(random_txt_tfidf)})

In [58]:
random_txt_tfidf_twit = tfidf2.transform(tmp['regrex']) #분석기 넘버 맞추기
clf2.predict(random_txt_tfidf_twit)

pred2 = pd.DataFrame({'clf2_label': clf2.predict(random_txt_tfidf_twit)})

In [59]:
pred_compare = pd.concat([pred1, pred2], axis=1)

In [60]:
pred_compare['comment'] = tmp.comment.to_list()

In [61]:
pred_compare

,clf1_label,clf2_label,comment
0,1,0,크
1,1,0,돌돌 말아서 보관 가능한가요?????
2,1,1,추카드려요♡
3,1,0,오히려 거리두기 끝난 요즘이 더 위험한거 같아요
4,1,0,굿모닝 입니다 잠시왔어요^^🤗
5,1,0,미세먼지땜에 마스크 해야되요 ㅎ
6,1,0,2022050229035281
7,0,0,성인 중형은 라이브혜택 포함 안되나용
8,1,0,안녕하세요~
9,1,0,ㅋㅋㅋㅋㅋㅋㅋ시상자가 더씬난


In [ ]:
# f = open('sample_file.txt', 'r')
# sentence = f.read()
# f.close()

In [ ]:
# print('형태소:', t.morphs(sentence))
# print('명사:', t.nouns(sentence))
# print('품사 태깅 결과:', t.pos(sentence))

# tokens_const = t.morphs(sentence)
# print('토큰의 수:', len(tokens_const))
# print(tokens_const[:100])